In [1]:
import sys
sys.path.append("../../common")

import deepctx as dc
from itertools import count
import matplotlib.pyplot as plt
import numpy as np
import re
import sympy as sp
from sympy import lambdify
from sympy.solvers import solve
from tqdm import tqdm
import tensorflow as tf
from IPython.display import clear_output


In [ ]:
dc.tf.devices.use(gpus=[])

In [2]:
with open("./puzzle_inputs/24.txt") as f:
    inp = f.read().rstrip()

In [3]:
hailstones = np.array([
    list(map(int, re.findall(r"-?\d+", line)))
    for line in inp.split('\n')]).reshape((-1, 2, 3)).astype(np.float64)

In [4]:
H = tf.constant(hailstones[:,0,:])
VH = tf.constant(hailstones[:,1,:])
# H, VH

2023-12-25 01:07:59.848660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-25 01:07:59.848881: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-25 01:07:59.877415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-25 01:07:59.877644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-25 01:07:59.877809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [5]:
P = tf.Variable(tf.random.normal((1, 3,), dtype=tf.float64), dtype=tf.float64)
VP = tf.Variable(tf.random.normal((1, 3,), dtype=tf.float64), dtype=tf.float64)
P, VP

(<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-0.59015289,  0.52826412,  0.25720672]])>,
 <tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[ 0.2999407 ,  0.91428485, -0.50910558]])>)

In [88]:
t = tf.reduce_mean((H - P) / (VP - VH), axis=1, keepdims=True)
t

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[ 13.63451989],
       [ 28.44870458],
       [ 12.52866965],
       [-12.21696902],
       [ -0.92224822]])>

In [87]:
d = P + t*VP - H - t*VH
d

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[-2.78920556e-01, -2.18283203e+01, -2.03623247e+01],
       [-5.22510259e+00,  1.88680862e+01, -2.79098124e+00],
       [-2.95869035e+00,  4.51917379e+00, -1.94676740e-02],
       [-2.03299723e+01, -5.86779705e+01, -2.28478259e+01],
       [-2.06236866e+01, -2.33380632e+01, -1.56896013e+01]])>

In [86]:
tf.losses.mean_squared_error(tf.zeros_like(H), d)

<tf.Tensor: shape=(5,), dtype=float64, numpy=
array([ 297.05921001,  130.36531698,    9.72571977, 1459.47838097,
        405.38840998])>

---

In [134]:
scale = 1/100000000
start = 200000000000000
end = 400000000000000

# scale = 1
# start = 7
# end = 27

In [158]:
@tf.function()
def step(P, VP):
    with tf.GradientTape() as tape:
        loss = compute_loss(P, VP)
    grads = tape.gradient(loss, [P, VP])
    optimizer.apply_gradients(zip(grads, [P, VP]))
    P.assign(tf.clip_by_value(P, start*scale, end*scale))
    VP.assign(tf.clip_by_value(VP, -1000*scale, 1000*scale))
    return loss

In [164]:
def compute_loss(P, VP):
    t = tf.reduce_max(tf.math.divide_no_nan((H - P), (VP - VH)), axis=1, keepdims=True)
    d = P + t*VP - H - t*VH
    loss = tf.losses.mean_absolute_error(tf.zeros_like(H), d)
    return loss

In [165]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)

In [166]:
P = tf.Variable([(start+end)//2*scale]*3, dtype=tf.float64)
VP = tf.Variable(tf.random.uniform((1, 3,), -1000*scale, 1000*scale, dtype=tf.float64), dtype=tf.float64)

In [167]:
P, VP

(<tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([3000000., 3000000., 3000000.])>,
 <tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-6.76781154e-06,  4.17407326e-06, -9.05998355e-06]])>)

In [168]:
for i in count():
    loss = step(P, VP)
    if i % 100 == 0:
        print(f"{i=};\n{P=}\n{VP=}\n{tf.reduce_sum(loss)=}")
        clear_output(wait=True)

KeyboardInterrupt: 

In [304]:
P, VP

(<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[23.97465265, 13.04729879, 10.03800169]])>,
 <tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-2.99302384,  0.98382107,  1.98522771]])>)

In [306]:
tf.round(P)

<tf.Tensor: shape=(1, 3), dtype=float64, numpy=array([[24., 13., 10.]])>

In [307]:
tf.round(VP)

<tf.Tensor: shape=(1, 3), dtype=float64, numpy=array([[-3.,  1.,  2.]])>

---

In [763]:
# scale = 1/100000000
# start = 200000000000000
# end = 400000000000000

scale = 1
start = 7
end = 27

In [764]:
with open("./puzzle_inputs/24_sample.txt") as f:
    inp = f.read().rstrip()

In [765]:
hailstones = np.array([
    list(map(int, re.findall(r"-?\d+", line)))
    for line in inp.split('\n')]).reshape((-1, 2, 3)).astype(np.float64)
H = tf.constant(hailstones[:,0,:])
VH = tf.constant(hailstones[:,1,:])

In [766]:
H, VH

(<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
 array([[19., 13., 30.],
        [18., 19., 22.],
        [20., 25., 34.],
        [12., 31., 28.],
        [20., 19., 15.]])>,
 <tf.Tensor: shape=(5, 3), dtype=float64, numpy=
 array([[-2.,  1., -2.],
        [-1., -1., -2.],
        [-2., -2., -4.],
        [-1., -2., -1.],
        [ 1., -5., -3.]])>)

In [822]:
def compute_p_for_times(H, VH, T):
    h1, h2 = H[:1], H[1:]
    vh1, vh2 = VH[:1], VH[1:]
    t1, t2 = T[:1], T[1:]
    Pn = ((t1*h2 - t2*h1) + t1*t2*(v2 - v1))/(t1-t2)
    return Pn

def compute_loss(Pn):
    loss = tf.zeros(Pn.shape[0], dtype=tf.float64)
    for P in Pn:
        loss += tf.losses.mean_absolute_error(P, Pn)
    return loss

@tf.function()
def step(T):
    with tf.GradientTape() as tape:
        Pn = compute_p_for_times(H, VH, T)
        loss = compute_loss(Pn)
    grads = tape.gradient(loss, [T])
    optimizer.apply_gradients(zip(grads, [T]))
    return loss

In [823]:
T = tf.Variable(tf.random.uniform((len(H), 1), dtype=tf.float64))
T

<tf.Variable 'Variable:0' shape=(5, 1) dtype=float64, numpy=
array([[0.84365346],
       [0.48348216],
       [0.80984814],
       [0.78075228],
       [0.46138681]])>

In [824]:
# T.assign([
#     [5], [3], [4], [6], [1.0]
# ])
# T

In [825]:
compute_p_for_times(H, VH, T)

<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
array([[ 17.79012552,  24.78921809,  11.26106175],
       [ 43.95623398, 251.8425288 ,  89.4034166 ],
       [-64.41480661, 223.00733152,  13.64700725],
       [ 24.26178561,  20.13223945,  -4.12291478]])>

In [826]:
Pn = compute_p_for_times(H, VH, T)
Pn

<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
array([[ 17.79012552,  24.78921809,  11.26106175],
       [ 43.95623398, 251.8425288 ,  89.4034166 ],
       [-64.41480661, 223.00733152,  13.64700725],
       [ 24.26178561,  20.13223945,  -4.12291478]])>

In [827]:
P = tf.reduce_mean(Pn, axis=0)
P

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([  5.39833463, 129.94282946,  27.5471427 ])>

In [828]:
optimizer = tf.keras.optimizers.SGD(1e-3)

In [829]:
for i in count():
    loss = step(T)
    if i % 100 == 0:
        P = tf.reduce_mean(compute_p_for_times(H, VH, T), axis=0)
        VP = tf.reduce_mean((H - P + T*VH)/T, axis=0)
        print(f"{i=};\n{P=}\n{VP=}\n{T=}\n{tf.reduce_sum(loss)=}")
        clear_output(wait=True)

KeyboardInterrupt: 

In [647]:
P = tf.reduce_mean(compute_p_for_times(H, VH, T), axis=0)
P

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([19.03554357, 12.88696428, 30.00881846])>

In [648]:
VP = tf.reduce_mean((H - P + T*VH)/T, axis=0)
VP

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-0.30687868, -4.38646791, -1.58132055])>

In [649]:
P + VP * T

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[19.04197912, 12.97895281, 30.0419803 ],
       [25.02236407, 98.4614842 , 60.85841074],
       [23.13898611, 71.5408232 , 51.15351942],
       [20.12476209, 28.45605505, 35.62147165],
       [20.57346402, 34.8697185 , 37.93359567]])>

In [650]:
H + VH * T

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[19.04194196, 12.97902902, 30.04194196],
       [37.50875322, 38.50875322, 61.01750644],
       [46.74309269, 51.74309269, 87.48618538],
       [15.54934565, 38.09869129, 31.54934565],
       [14.98850677, 44.05746615, 30.03447969]])>

---

In [640]:
# scale = 1/100000000
# start = 200000000000000
# end = 400000000000000

scale = 1
start = 7
end = 27

In [641]:
with open("./puzzle_inputs/24_sample.txt") as f:
    inp = f.read().rstrip()

In [565]:
hailstones = np.array([
    list(map(int, re.findall(r"-?\d+", line)))
    for line in inp.split('\n')]).reshape((-1, 2, 3)).astype(np.float64)
H = tf.constant(hailstones[:,0,:]) * scale
VH = tf.constant(hailstones[:,1,:]) * scale

In [566]:
# H, VH

In [567]:
P = tf.Variable(tf.random.uniform((1, 3), start, end, dtype=tf.float64))
P

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[10.20772451, 19.28932548, 10.85541575]])>

In [568]:
VP = tf.Variable(tf.random.uniform((1, 3), dtype=tf.float64))
VP

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[0.81790247, 0.46203896, 0.52150204]])>

In [569]:
h = sp.Symbol("h")
p = sp.Symbol("p")
vh = sp.Symbol("v_h")
vp = sp.Symbol("v_p")
t = sp.Symbol("t")

f = (h + t*vh - p - t*vp)**2
df_dp = f.diff(p)
df_dvp = f.diff(vp)
distance = lambdify([h, p, vh, vp, t], f)
p_grad = lambdify([h, p, vh, vp, t], df_dp)
vp_grad = lambdify([h, p, vh, vp, t], df_dvp)

In [570]:
h = sp.IndexedBase("h")
p = sp.IndexedBase("p")
vh = sp.IndexedBase("v_h")
vp = sp.IndexedBase("v_p")
t = sp.Symbol("t")

f = (h + t*vh - p - t*vp)**2
distance = lambdify([h, p, vh, vp, t], f)

# df_dp = f.diff(p)
# df_dvp = f.diff(vp)

# time optimization
df_dt = lambdify([h, p, vh, vp, t], f.diff(t))
df_dt_components = sum(df_dt(h[i], p[i], vh[i], vp[i], t) for i in range(3))
t_eq = lambdify([h, p, vh, vp], solve(df_dt_components, t)[0])
optimize_t = lambda h, p, vh, vp: tf.expand_dims(t_eq(tf.transpose(h), tf.transpose(p), tf.transpose(vh), tf.transpose(vp)), axis=1)

In [571]:
optimize_t(H, P, VH, VP)

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[5.23941339],
       [3.54598477],
       [4.24725816],
       [4.98022239],
       [0.26572594]])>

In [585]:
p_grad(H, P, VH, VP, optimize_t(H, P, VH, VP))

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[ 11.94376092,   6.94145045, -11.86678538],
       [ -2.692042  ,  10.94738677,  -4.40675281],
       [  4.35216756,   9.49248112,  -7.88119562],
       [ 14.52256621,   1.1016541 , -19.13433144],
       [-19.68132705,   3.48146187,  -6.4176596 ]])>

In [584]:
vp_grad(H, P, VH, VP, optimize_t(H, P, VH, VP))

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[ 62.5783009 ,  36.36912843, -62.17499421],
       [ -9.54593995,  38.81926681, -15.62627835],
       [ 18.48477918,  40.31701788, -33.47347242],
       [ 72.32560937,   5.48648241, -95.29322581],
       [ -5.22983918,   0.92511474,  -1.70533865]])>

In [583]:
distance(H, P, VH, VP, optimize_t(H, P, VH, VP))

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[35.66335623, 12.04593358, 35.2051488 ],
       [ 1.81177254, 29.96131928,  4.85486757],
       [ 4.73534062, 22.52679944, 15.52831112],
       [52.72623233,  0.30341044, 91.5306599 ],
       [96.83865865,  3.03014419, 10.29658868]])>

In [593]:
h = H.numpy()
vh = VH.numpy()
p = P.numpy()
vp = VP.numpy()

In [639]:
for i in range(100):
    d = distance(h, p, vh, vp, optimize_t(h, p, vh, vp))
    t = optimize_t(h, p, vh, vp)
    next_p = d / p_grad(h, p, vh, vp, t)
    next_vp = d / vp_grad(h, p, vh, vp, t)
    if np.any(np.isnan(next_p)) or np.any(np.isnan(next_vp)):
        print("Found nan")
        break
    p = np.sum(next_p, axis=0)
    vp = np.sum(next_vp, axis=0)
p, vp

(array([-12.90479145,   5.87819318,   2.42957598]),
 array([-3.90347277,  1.73140454,  0.77005902]))

In [576]:
p_grad(h, p, vh, vp, t)

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[-192.54500868,   31.8189045 , -151.28056891],
       [-146.32177766,   20.48082532, -113.81065921],
       [-164.48017797,   25.38009934, -129.71262801],
       [-180.65130587,   23.63060008, -153.954895  ],
       [ -45.42396575,  -16.70725133,  -57.69088159]])>

In [472]:
df_dt(h[0], p, vh[0], vp, t) + df_dt(h[1], p, vh[1], vp, t) + df_dt(h[2], p, vh[2], vp, t)

(2*v_h[0] - 2*v_p)*(t*v_h[0] - t*v_p + h[0] - p) + (2*v_h[1] - 2*v_p)*(t*v_h[1] - t*v_p + h[1] - p) + (2*v_h[2] - 2*v_p)*(t*v_h[2] - t*v_p + h[2] - p)

In [382]:
sum(df_dt(h[i], p, vh[i], vp, t) for i in range(3))

TypeError: 'Add' object is not callable

In [380]:
f

(t*v_h - t*v_p + h - p)**2

In [359]:
solve(df_dt, t)[0]

(-h + p)/(v_h - v_p)

In [360]:
t_eq(H, P, VH, VP)

<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[5., 0., 5.],
       [3., 3., 3.],
       [4., 4., 4.],
       [6., 6., 6.],
       [1., 1., 1.]])>

In [350]:
distance(H[0], P[0], VH[0], VP[0], 5.0)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0., 0., 0.])>

In [325]:
p_grad(H[0], P[0], VH[0], VP[0], 5.0)

<tf.Tensor: shape=(), dtype=float64, numpy=0.0>

In [324]:
v_grad(H[0], P[0], VH[0], VP[0], 5.0)

<tf.Tensor: shape=(), dtype=float64, numpy=0.0>

In [311]:
P.assign([[24, 13, 10]])
VP.assign([[-3, 1, 2]])

<tf.Variable 'UnreadVariable' shape=(1, 3) dtype=float64, numpy=array([[-3.,  1.,  2.]])>

In [312]:
optimal_t(H[0], P[0], VH[0], VP[0])

<tf.Tensor: shape=(), dtype=float64, numpy=5.0>

In [303]:
P[0]

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([3.93736453e+14, 2.78205093e+14, 2.98742186e+14])>

In [298]:
distance(h, p, vh, vp, t).diff(p[0])
distance(h, p, vh, vp, t).diff(p[0])
distance(h, p, vh, vp, t).diff(p[0])

-2*t*v_h[0] + 2*t*v_p[0] - 2*h[0] + 2*p[0]

In [668]:
# min_distance(H, P, VH, VP)

In [675]:
optimizer = tf.keras.optimizers.Adam(1e-2)

In [676]:
@tf.function()
def step(P, VP):
    with tf.GradientTape() as tape:
        y_pred = tf.reduce_sum(min_distance(H, P, VH, VP))
        loss = tf.keras.losses.mean_absolute_error([0.0], y_pred)
    grads = tape.gradient(loss, [P, VP])
    optimizer.apply_gradients(zip(grads, [P, VP]))
    return loss

In [682]:
def go():
    for i in count():
        loss = step(P, VP)
        if i % 100 == 0:
            print(f"{i=};\n{P=}\n{VP=}\n{tf.reduce_sum(loss)=}")
            clear_output(wait=True)
        if loss < 1e-3:
            break

In [704]:
P = tf.Variable(tf.random.uniform((1, 3), start, end, dtype=tf.float64))
VP = tf.Variable(tf.random.normal((1, 3), dtype=tf.float64))

In [705]:
P

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[14.75658668, 19.30516533, 21.46618938]])>

In [706]:
VP

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-0.21087257, -0.65685728,  1.74092709]])>

In [707]:
go()

i=1100;
P=<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[23.88191231, 13.16915516, 10.16466116]])>
VP=<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-2.97467574,  0.96362812,  1.96607602]])>
tf.reduce_sum(loss)=<tf.Tensor: shape=(), dtype=float64, numpy=0.0014270412298931416>


In [708]:
tf.round(P)

<tf.Tensor: shape=(1, 3), dtype=float64, numpy=array([[24., 13., 10.]])>

In [703]:
tf.round(VP)

<tf.Tensor: shape=(1, 3), dtype=float64, numpy=array([[-3.,  1.,  2.]])>

---

In [6]:
# min_distance(H, P, VH, VP)

In [404]:
# scale = 100000000000
# scale = 0.0001
scale = 1

In [405]:
start = 200000000000000 / scale
end = 400000000000000 / scale

# start = 200000000000000
# end = 400000000000000

# start=tf.constant(7.0, dtype=tf.float64)
# end=tf.constant(27.0, dtype=tf.float64)
start, end

(200000000000000.0, 400000000000000.0)

In [264]:
with open("./puzzle_inputs/24.txt") as f:
    inp = f.read().rstrip()

In [265]:
hailstones = np.array([
    list(map(int, re.findall(r"-?\d+", line)))
    for line in inp.split('\n')]).reshape((-1, 2, 3)).astype(np.float64)

In [266]:
H = tf.constant(hailstones[:,0,:]) / scale
VH = tf.constant(hailstones[:,1,:])
# VH

In [267]:
h = sp.IndexedBase("h")
p = sp.IndexedBase("p")
vh = sp.IndexedBase("v_h")
vp = sp.IndexedBase("v_p")
t = sp.Symbol("t")

f = (h + t*vh - p - t*vp)**2
distance = lambdify([h, p, vh, vp, t], f)

# df_dp = f.diff(p)
# df_dvp = f.diff(vp)

# time optimization
df_dt = lambdify([h, p, vh, vp, t], f.diff(t))
df_dt_components = sum(df_dt(h[i], p[i], vh[i], vp[i], t) for i in range(3))
t_eq = lambdify([h, p, vh, vp], solve(df_dt_components, t)[0])
optimize_t = lambda h, p, vh, vp: tf.expand_dims(t_eq(tf.transpose(h), tf.transpose(p), tf.transpose(vh), tf.transpose(vp)), axis=1)


In [268]:
P = tf.Variable(tf.random.uniform((1, 3), start, end, dtype=tf.float64))
VP = tf.Variable(tf.random.normal((1, 3), dtype=tf.float64))

In [316]:
tf.keras.backend.set_epsilon(1e-25)

In [406]:
P.assign([[461522278379691.0 / scale, 278970483473653.0 / scale, 243127954482398.0 / scale]])
VP.assign([[-336, 29, 38.0]])

<tf.Variable 'UnreadVariable' shape=(1, 3) dtype=float64, numpy=array([[-336.,   29.,   38.]])>

In [419]:
tf.keras.backend.epsilon
optimizer = tf.keras.optimizers.SGD(1e-12)
# optimizer = tf.keras.optimizers.Adam(1e-1)

@tf.function()
def step(P, VP):
    with tf.GradientTape() as tape:
        y_pred = tf.reduce_sum(distance(H, P, VH, VP, optimize_t(H, P, VH, VP)))
        loss = tf.keras.losses.mean_absolute_error([0.0], y_pred)
    # grads = tape.gradient(loss, [P, VP])
    # optimizer.apply_gradients(zip(grads, [P, VP]))
    grads = tape.gradient(loss, [P])
    grads = tf.cast(-1e-25*(grads[0]) * P, dtype=tf.int64)
    # P.assign_add(-1e-25*(grads[0] * P))
    # optimizer.apply_gradients(zip(grads, [P]))
    return loss, grads

In [420]:
loss, grads = step(P, VP)
print(f"{list(map(int, P.numpy().flatten()*scale))=}\n{VP=}\n{tf.reduce_sum(loss)=}\n{grads=}")
# print(f"{-1e-25*(grads[0] * P)}")

list(map(int, P.numpy().flatten()*scale))=[461522278379691, 278970483473653, 243127954482398]
VP=<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-336.,   29.,   38.]])>
tf.reduce_sum(loss)=<tf.Tensor: shape=(), dtype=float64, numpy=6.189780918523508e+39>
grads=<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[51017974487, 39640529519, 33149495164]])>


In [403]:
delta = (1e-25*grads[0] * P).numpy().astype(np.float128)
p = (P.numpy().astype(np.float128) + delta).astype(np.float64)
# P.numpy().astype(np.float128) + delta.numpy()
print(f"{list(map(int, p.flatten()))=}\n{delta=}")

list(map(int, p.flatten()))=[4615222783796910080, 2789704834736529920, 2431279544823980032]
delta=array([[-32.04425829,  15.399607  ,  14.98889485]], dtype=float128)


In [385]:
delta = 1e-25*grads[0] * P
P.assign_add(delta)
print(f"{list(map(int, P.numpy().flatten()))=}\n{delta=}")

list(map(int, P.numpy().flatten()))=[4615222783796910080, 2789704834736529920, 2431279544823980032]
delta=<tf.Tensor: shape=(1, 3), dtype=float64, numpy=array([[-32.04425829,  15.399607  ,  14.98889485]])>


In [344]:
X = tf.Variable(tf.random.uniform((1, 3), start, end, dtype=tf.float64))
X.assign(P)

<tf.Variable 'UnreadVariable' shape=(1, 3) dtype=float64, numpy=array([[4.61522278e+18, 2.78970483e+18, 2.43127954e+18]])>

In [354]:
delta = 1e-25*grads[0] * X

In [355]:
delta

<tf.Tensor: shape=(1, 3), dtype=float64, numpy=array([[-32.04425829,  15.399607  ,  14.98889485]])>

In [360]:
tf.math.mod(X, 10000000)

2023-12-25 01:40:36.466044: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: JIT compilation failed.


UnknownError: JIT compilation failed. [Op:FloorMod]

In [346]:
X.assign_add(1e-25*grads[0] * X)
print(f"{list(map(int, X.numpy().flatten()))=}")

list(map(int, X.numpy().flatten()))=[4615222783796910080, 2789704834736529920, 2431279544823980032]


In [121]:
tf.keras.backend.set_epsilon(1e-20)

In [122]:
tf.keras.losses.mean_absolute_error([0.0], tf.reduce_sum(distance(H, P, VH, VP, optimize_t(H, P, VH, VP))))

<tf.Tensor: shape=(), dtype=float64, numpy=322242600.78100586>

In [108]:
def go():
    for i in count():
        loss, grads = step(P, VP)
        if i % 100 == 0:

            print(f"{i=};\n{list(map(int, P.numpy().flatten()*scale))=}\n{VP=}\n{tf.reduce_sum(loss)=}\n{grads=}")
            clear_output(wait=True)
        if loss < -10:
            break

In [109]:
P

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[4.61522278e+14, 2.78970483e+14, 2.43127954e+14]])>

In [110]:
VP

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[-336.,   29.,   38.]])>

In [178]:
go()

KeyboardInterrupt: 

In [ ]:
P, VP

In [1079]:
end

400.0

In [1080]:
list(map(int, tf.round(P*1000000000000).numpy().flatten()))[0] > 400000000000000

True

In [1081]:
P*1000000000000

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float64, numpy=array([[461.52227838, 278.97048347, 243.12795448]])>

In [1629]:
slope = VP.numpy()
slope

array([[-336.,   29.,   38.]])

In [1633]:
first_hit_index = np.argmin(optimize_t(H, P, VH, VP))
first_hit_index

17

In [342]:
found_P = (P.numpy().flatten()*scale).astype(np.int64)

In [343]:
found_P

array([461522278379691, 278970483473653, 243127954482398])

In [58]:
sum(found_P)

983620716334964

In [1624]:
found_P

[461522278379867, 278970483473622, 243127954482353]

In [1369]:
found_P

[461522278379589, 278970483473672, 243127954482411]

In [1650]:
found_VP = VP.numpy().astype(np.int64)

In [1653]:
H_np = H.numpy().astype(np.int64)
VH_np = VH.numpy().astype(np.int64)

for x in range(-1000, 1000+1):
    for y in range(-1000, 1000+1):
        for z in range(-1000, 1000+1):
            new_p = found_P + [x, y, z]
            t = optimize_t(H_np, new_p, VH_np, found_VP)
            m = np.max(distance(H_np, new_p, VH_np, found_VP, t))
        break
        print(m)
    break

In [1259]:
# 983620716335745 - wrong
# 983620716334964
# 983620716334761 - too low

In [1641]:
t = optimize_t(H, P, VH, VP)
# t

In [1642]:
# distance(H*1000000000000, P*1000000000000, VH*1000000000000, VP*1000000000000, t)